### Assignement 2
Name: Zheming Kang; uid: 3036195746

### Task1: Index development

In [86]:
# Load the files and add index column
import pandas as pd
filename = "Beijing_restaurants.txt"
data = pd.read_csv(filename, header = None, skiprows = 1, delimiter=" ", names = ["x", "y"])

data["id"] = range(1, len(data)+1)
print(data)

               x           y     id
0      39.856138  116.423940      1
1      39.813336  116.486149      2
2      39.999318  116.340329      3
3      39.925426  116.407166      4
4      40.089314  116.439558      5
...          ...         ...    ...
51965  39.882149  116.315335  51966
51966  39.965801  116.659559  51967
51967  39.865790  116.403969  51968
51968  39.935663  116.396346  51969
51969  39.932615  116.435572  51970

[51970 rows x 3 columns]


In [87]:
# Calculate min max values
x_min = data["x"].min()
x_max = data["x"].max()

y_min = data["y"].min()
y_max = data["y"].max()

print("x min is: ", x_min)
print("x max is: ", x_max)
print("y min is: ", y_min)
print("y max is: ", y_max)

x min is:  39.68009
x max is:  40.179911
y min is:  116.070466
y max is:  116.719976


In [88]:
# Calculate the interval
x_interval = (x_max - x_min) / 10
y_interval = (y_max - y_min) / 10
print("x interval is: ", x_interval)
print("y interval is: ", y_interval)

x interval is:  0.04998209999999972
y interval is:  0.06495100000000065


In [89]:
# Assign point to grid
data["x_grid"] = (data["x"] - x_min) // x_interval
data["y_grid"] = (data["y"] - y_min) // y_interval
data["x_grid"] = data["x_grid"].clip(0, 9)
data["y_grid"] = data["y_grid"].clip(0, 9)
# Convert to integer   
data["x_grid"] = data["x_grid"].astype(int)
data["y_grid"] = data["y_grid"].astype(int)


In [90]:
# Sort the datapoints by grid
data = data.sort_values(by=["x_grid", "y_grid"]).reset_index(drop=True)
print(data)


               x           y     id  x_grid  y_grid
0      39.729270  116.119278     56       0       0
1      39.729398  116.128704    573       0       0
2      39.723127  116.121828   1253       0       0
3      39.729585  116.127883   1372       0       0
4      39.729571  116.128738   1395       0       0
...          ...         ...    ...     ...     ...
51965  40.137399  116.660384  43673       9       9
51966  40.132776  116.657197  43799       9       9
51967  40.130205  116.669393  44767       9       9
51968  40.136146  116.656784  45984       9       9
51969  40.135022  116.662107  47573       9       9

[51970 rows x 5 columns]


In [91]:
# Store the grid information into grid.grd file
grid_file = open("grid.grd", "w")
for point in data.itertuples():
    x_str = "{:.6f}".format(point.x)  # 保留 6 位小数
    y_str = "{:.6f}".format(point.y)  # 保留 6 位小数
    grid_file.write(str(point.id) + " " + x_str + " " + y_str + "\n")

In [92]:
# Count the number of points in each grid
grid_count = data.groupby(["x_grid", "y_grid"]).size().reset_index(name="count")
print(grid_count)

    x_grid  y_grid  count
0        0       0    108
1        0       1    179
2        0       2     20
3        0       3    117
4        0       4    356
..     ...     ...    ...
93       9       5    130
94       9       6     74
95       9       7     69
96       9       8    148
97       9       9     28

[98 rows x 3 columns]


In [93]:
# Define a function that can compute the offset in the grid.grd file
# Parameters: file_path: the file path of the grid.grd file
#             n: the line number
def get_line_offset(file_path, n):
    with open(file_path, 'r') as file:
        offset = 0
        line_number = 0
        for line in file:
            if line_number == n:
                return offset
            offset += len(line)
            line_number += 1
    return -1 

In [94]:
# Get the offset of each grid in the grid.grd file
offsets = [0]
index = 0
for point in grid_count.itertuples():
    index += grid_count["count"][point.Index]
    # print(index)
    offsets.append(get_line_offset("grid.grd", index))
print(offsets)

[0, 2894, 7688, 8226, 11368, 20915, 23352, 24907, 25393, 27612, 38826, 41643, 50004, 70305, 76720, 79055, 87547, 88563, 88748, 89042, 96795, 99639, 112137, 131472, 151539, 168964, 175648, 177176, 177628, 177789, 182011, 195518, 244359, 298950, 360859, 395500, 404847, 410154, 417839, 421002, 433829, 461389, 508543, 614771, 731546, 770605, 788900, 797622, 813912, 824793, 829850, 846128, 899422, 983825, 1070669, 1115355, 1127657, 1130209, 1131976, 1132246, 1133531, 1140280, 1163957, 1192674, 1224366, 1248929, 1251663, 1255548, 1256540, 1257721, 1261453, 1270742, 1279180, 1292885, 1300839, 1303015, 1311310, 1319940, 1321304, 1322460, 1323748, 1329163, 1338592, 1341980, 1350208, 1352086, 1356727, 1363292, 1367043, 1367177, 1370443, 1373540, 1377363, 1380038, 1383521, 1385504, 1387356, 1391335, -1]


In [95]:
# Store the grid_count information in grid.dir file
grid_dir = open("grid.dir", "w")
grid_dir.write(str(x_min) + " " + str(x_max) + " " + str(y_min) + " " + str(y_max) + "\n")
i = 0
for grid in grid_count.itertuples():
    grid_dir.write(str(grid.x_grid) + " " + str(grid.y_grid) + " "+str(offsets[i]) +" "+ str(grid.count) + "\n")
    i += 1

### Task 2: range selection queries

In [96]:
# Task 2: Range Selection Queries
def range_selection(x_low, x_high, y_low, y_high, grid_dir, grid_grd):
    result = []
    with open(grid_dir, "r") as file:
        firstline = file.readline()  
        x_min, x_max, y_min, y_max = firstline.split()
        x_max = float(x_max)
        x_min = float(x_min)
        y_max = float(y_max)
        y_min = float(y_min)
        x_interval = (x_max - x_min) / 10
        y_interval = (y_max - y_min) / 10
        # print("xmin = ", x_min, "xmax = ", x_max, "ymin = ", y_min, "ymax = ", y_max, "x_interval = ", x_interval, "y_interval = ", y_interval)
        previous_cnt = 0
        for line in file:
            x, y, offset, count = line.split()
            x = int(x)
            y = int(y)
            offset = int(offset)
            count = int(count)
            # If the grid is entirely within the range
            # print("x = ", x, "x*x_interval+x_min = ", x*x_interval+x_min, "y = ", y, "y*y_interval+y_min = ", y*y_interval+y_min)
            if x_low <= x*x_interval+x_min and x*x_interval+x_min+x_interval<=x_high and y_low <= y*y_interval+y_min and y*y_interval+y_min+y_interval<=y_high:
                with open(grid_grd, "r") as file:
                    file.seek(offset+previous_cnt)
                    for i in range(count):
                        line = file.readline()
                        # print("Case1, ", line)
                        id, x, y = line.split()
                        x = float(x)
                        y = float(y)
                        result.append([x, y])
            # If the grid is partially within the range
            elif x_low <= x*x_interval+x_min<=x_high or x_low <= x*x_interval+x_min+x_interval<=x_high or y_low <= y*y_interval+y_min<=y_high or y_low <= y*y_interval+y_min+y_interval<=y_high:
                with open(grid_grd, "r") as file:
                    file.seek(offset+previous_cnt)
                    for i in range(count):
                        line = file.readline()
                        # print("Case2, ", line)
                        id, x, y = line.split()
                        x = float(x)
                        y = float(y)
                        if x_low <= x <= x_high and y_low <= y <= y_high:
                            # result.append(line)
                            result.append([x, y])
            previous_cnt += count

            
    return result

In [ ]:
import argparse

# 创建命令行参数解析器
parser = argparse.ArgumentParser(description='Range Selection Queries')

# 添加命令行参数
parser.add_argument('x_low', type=float, help='x_low value')
parser.add_argument('x_high', type=float, help='x_high value')
parser.add_argument('y_low', type=float, help='y_low value')
parser.add_argument('y_high', type=float, help='y_high value')
parser.add_argument('grid_dir', type=str, help='grid_dir value')
parser.add_argument('grid_grd', type=str, help='grid_grd value')

# 解析命令行参数
args = parser.parse_args()

# 使用参数调用函数
result = range_selection(args.x_low, args.x_high, args.y_low, args.y_high, args.grid_dir, args.grid_grd)

# 打印结果
print(len(result))

In [97]:
result = range_selection(x_min, x_max, y_min, y_max, "grid.dir", "grid.grd")
print(len(result))

51970


In [98]:
# Check the correctness
# Set x_low = 39.70, x_high = 39.80, y_low = 116.13, y_high = 116.26
result = range_selection(39.70, 39.80, 116.13, 116.26, "grid.dir", "grid.grd")
print(len(result))



777


In [99]:
# Conver result to dataframe
# result = pd.DataFrame(result, columns=["x", "y"])
# # sort the result by x and y
# result = result.sort_values(by=["x", "y"]).reset_index(drop=True)
# print(result)

In [100]:
# Count the number of points in the range from Beijing_restaurants.txt
count = 0
result = []
# Read the file
with open("Beijing_restaurants.txt", "r") as file:
    # skip line 1
    file.readline()
    for line in file:
        x, y = line.split()
        x = float(x)
        y = float(y)
        if 39.70 <= x <= 39.80 and 116.13 <= y <= 116.26:
            count += 1
            # result.append(line)
print(count)
# print(result[0:10])

777


### Task 3: nearest neighbor queries

In [101]:
# Task 3: Nearest Neighbor Queries
# 1. Distance computation for points
def distance_point(x1, y1, x2, y2):
    return ((x1-x2)**2 + (y1-y2)**2)**0.5

# 2. Distance computation for grids
def distance_grid(x_point, y_point, x_grid, y_grid,x_min, y_min, x_interval, y_interval):
    x_low = x_min+x_grid*x_interval
    x_high = x_min+(x_grid+1)*x_interval
    y_low = y_min+y_grid*y_interval
    y_high = y_min+(y_grid+1)*y_interval
    x_distance = min(abs(x_point-x_low), abs(x_point-x_high))
    y_distance = min(abs(y_point-y_low), abs(y_point-y_high))
    if x_low <= x_point <= x_high and y_low <= y_point <= y_high:
        return 0
    elif x_low <= x_point <= x_high:
        return y_distance
    elif y_low <= y_point <= y_high:
        return x_distance
    else:
        return (x_distance**2 + y_distance**2)**0.5
    return -1


In [102]:
# 3. Nearest Neighbor Queries
import heapq as hq

# 3.1 Convert the point to grid
def convert_to_grid(x, y, grid_dir):
    # Read the first line and get the min and max values
    x_grid = -1
    y_grid = -1
    with open(grid_dir, "r") as file:
        firstline = file.readline()
        x_min, x_max, y_min, y_max = firstline.split()
        x_min = float(x_min)
        x_max = float(x_max)
        y_min = float(y_min)
        y_max = float(y_max)
        x_grid = (x - x_min) // x_interval
        y_grid = (y - y_min) // y_interval
    return x_grid, y_grid


def nearest_neighbor(target_x, target_y, grid_dir, grid_grd):
    pq = []
    visited_grid = set()
    visited_point = set()
    gx, gy = convert_to_grid(target_x, target_y, grid_dir)
    gx = int(gx)
    gy = int(gy)
    hq.heappush(pq, (0, gx, gy, True)) # pq: (distance, x, y, isGrid)
    visited_grid.add((gx, gy))
    while(len(pq)>0):
        distance, x, y, isGrid = hq.heappop(pq)
        if x==target_x and y==target_y:
            continue # Skip the current point
        if isGrid:# If the first element is a grid, put all the points into the queue
            x = int(x)
            y = int(y)
            # First, we need to get the direction of the points by grid.dir file
            # Skipping the first line, read the grid.dir file and convert to dataframe
            grid_dir = pd.read_csv("grid.dir", delimiter=" ", header=None, skiprows=1, names=["x_grid", "y_grid", "offset", "count"])
            # Find the position of the grid in the grid.dir file
            # grid = grid_dir[(grid_dir["x_grid"]==x) & (grid_dir["y_grid"]==y)]
            grid = grid_dir[(grid_dir["x_grid"] == x) & (grid_dir["y_grid"] == y)]
            # print(grid)
            if len(grid) == 0:
                print("x = ", target_x, "y = ", target_y, "is not in the grid")
                return -1
            # Count the sum of the previous points
            previous_cnt = grid_dir[grid_dir["x_grid"]<x]["count"].sum()
            previous_cnt += grid_dir[(grid_dir["x_grid"]==x) & (grid_dir["y_grid"]<y)]["count"].sum()
            # Read the grid.grd file and get the points
            with open("grid.grd", "r") as file:
                # print(len(grid["offset"]))
                offset = grid["offset"].item() + previous_cnt
                # print("grid[\"offset\"] is ", grid["offset"].item())

                # print(offset)
                file.seek(offset)
                for i in range(grid["count"].item()):
                    line = file.readline()
                    id, px, py = line.split()
                    px = float(px)
                    py = float(py)
                    if((px, py) not in visited_point):
                        hq.heappush(pq, (distance_point(target_x, target_y, px, py), px, py, False))
                        visited_point.add((px, py))
            # Then, add all the adjecent grids into the queue
            for i in range(-1, 2):
                for j in range(-1, 2):
                    if(0<=x+i<10 and 0<=y+j<10 and (x+i, y+j) not in visited_grid):
                        hq.heappush(pq, (distance_grid(target_x, target_y, x+i, y+j, x_min, y_min, x_interval, y_interval), x+i, y+j, True))
                        visited_grid.add((x+i, y+j))
        else: # If the first element is a point, return the point
            yield (x, y, distance)                  

            

In [103]:
k = 3
result = []
x, y = 39.856138, 116.42394
ite = nearest_neighbor(x, y, "grid.dir", "grid.grd")
for i in range(k):
    nn = next(ite)
    if nn == -1:
        break
    result.append(nn)

for i in range(len(result)):
    gx, gy = convert_to_grid(result[i][0], result[i][1], "grid.dir")
    print("The nearest neighbor of point (", x, ", ", y, ") is (", result[i][0], ", ", result[i][1], ") in grid (", gx, ", ", gy, "), distance is ", result[i][2])

The nearest neighbor of point ( 39.856138 ,  116.42394 ) is ( 39.855923 ,  116.424059 ) in grid ( 3.0 ,  5.0 ), distance is  0.00024573563030494085
The nearest neighbor of point ( 39.856138 ,  116.42394 ) is ( 39.856008 ,  116.42362 ) in grid ( 3.0 ,  5.0 ), distance is  0.00034539832078483794
The nearest neighbor of point ( 39.856138 ,  116.42394 ) is ( 39.855681 ,  116.423581 ) in grid ( 3.0 ,  5.0 ), distance is  0.0005811454207048572


In [104]:
x, y = 39.856138, 116.42394
gx, gy = convert_to_grid(x, y, "grid.dir")
print("The grid of point (", x, ", ", y, ") is (", gx, ", ", gy, ")")

The grid of point ( 39.856138 ,  116.42394 ) is ( 3.0 ,  5.0 )


In [105]:
x, y = 39.729270, 116.119278
gx, gy = convert_to_grid(x, y, "grid.dir")
print("The grid of point (", x, ", ", y, ") is (", gx, ", ", gy, ")")

k = 110
result = []
ite = nearest_neighbor(x, y, "grid.dir", "grid.grd")
for i in range(k):
    nn = next(ite)
    if nn == -1:
        break
    result.append(nn)

for i in range(len(result)):
    gx, gy = convert_to_grid(result[i][0], result[i][1], "grid.dir")
    print("The nearest neighbor of point (", x, ", ", y, ") is (", result[i][0], ", ", result[i][1], ") in grid (", gx, ", ", gy, "), distance is ", result[i][2])

The grid of point ( 39.72927 ,  116.119278 ) is ( 0.0 ,  0.0 )
The nearest neighbor of point ( 39.72927 ,  116.119278 ) is ( 39.729335 ,  116.119371 ) in grid ( 0.0 ,  0.0 ), distance is  0.00011346365057220968
The nearest neighbor of point ( 39.72927 ,  116.119278 ) is ( 39.729336 ,  116.119371 ) in grid ( 0.0 ,  0.0 ), distance is  0.00011403946686027507
The nearest neighbor of point ( 39.72927 ,  116.119278 ) is ( 39.72981 ,  116.11914 ) in grid ( 0.0 ,  0.0 ), distance is  0.0005573544653081311
The nearest neighbor of point ( 39.72927 ,  116.119278 ) is ( 39.729649 ,  116.119817 ) in grid ( 0.0 ,  0.0 ), distance is  0.00065890970550254
The nearest neighbor of point ( 39.72927 ,  116.119278 ) is ( 39.729734 ,  116.11993 ) in grid ( 0.0 ,  0.0 ), distance is  0.0008002499609521169
The nearest neighbor of point ( 39.72927 ,  116.119278 ) is ( 39.728809 ,  116.119954 ) in grid ( 0.0 ,  0.0 ), distance is  0.0008182279633565116
The nearest neighbor of point ( 39.72927 ,  116.119278 ) i